In [5]:
cd("C:/Users/spatial stat/Desktop/2019-2/고급통계계산/Project/github/AdvStatComp_HDP/src")
include("hchol_sh.jl")
β = 0.3
ns = [16, 32, 64, 128]
#ns = [4, 8, 16, 32, 64]
chol_time = zeros(length(ns))
potrf_time = zeros(length(ns))
hchol_time = zeros(length(ns))
hchol_error = zeros(length(ns))
for i in 1:length(ns)
    n = ns[i]
    N = n * n
    Σ = zeros(N, N)
    #Exponential Covariance Matrix with β
    #n points evenly distributed on a grid in the unit square and indexed with Morton's order
    function morton(n::Int64)
        if n == 1
            return [1], [1]
        end
        a, b = morton(n÷2)
        return vcat(a, a .+ n÷2, a, a .+ n÷2), vcat(b, b, b .+ n÷2, b .+ n÷2)
    end

    a, b = morton(n)
    for i in 1:N
        for j in 1:N
            Σ[i, j] = exp( -norm([a[i] - a[j], b[i] - b[j]], 2) / β)
        end
    end
    Σ = Symmetric(Σ)
    Σ1 = copy(Σ)
    chol_time[i] = @elapsed cholesky(Σ)
    potrf_time[i] = @elapsed LAPACK.potrf!('L', Matrix(Σ))[1]
    hchol_time[i] = @elapsed L = hchol(Σ, n)
    hchol_error[i] =norm(uncompress(L, n) - Σ1, 2)/norm(Σ1, 2)
end
println(ns.^2)
println(chol_time)
println(potrf_time)
println(hchol_time)
println(hchol_error)

[256, 1024, 4096, 16384]
[0.001621999, 0.0096584, 0.414299701, 156.3175917]
[0.0007207, 0.0132376, 0.430535101, 154.0718618]
[0.1532465, 0.0762115, 0.9161763, 37.3329991]
[1.060943412202425e-7, 9.966979869024319e-7, 0.0011113011849685343, 0.0018707146975361433]


In [12]:
println(chol_time)
println(potrf_time)
println(hchol_time)
println(hchol_error)

[0.007150199, 0.1849618, 1.066223699, 142.725039201]
[0.008398399, 0.049058399, 0.917584099, 156.701292201]
[0.1777536, 0.0460408, 1.4873949, 25.1979474]
[0.042844201, 0.1009667, 2.656264101, 0.0]


In [23]:
cd("C:/Users/spatial stat/Desktop/2019-2/고급통계계산/Project/github/AdvStatComp_HDP/src")
include("hchol_sh.jl")
using LinearAlgebra
β = 0.3
n = 128
N = n * n
Σ = zeros(N, N)
#Exponential Covariance Matrix with β
#n points evenly distributed on a grid in the unit square and indexed with Morton's order
function morton(n::Int64)
    if n == 1
        return [1], [1]
    end
    a, b = morton(n÷2)
    return vcat(a, a .+ n÷2, a, a .+ n÷2), vcat(b, b, b .+ n÷2, b .+ n÷2)
end

a, b = morton(n)
for i in 1:N
    for j in 1:N
        Σ[i, j] = exp( -norm([a[i] - a[j], b[i] - b[j]], 2) / β)
    end
end

#@elapsed hchol_iter(Σ, n)

In [24]:
cholesky(Σ)

Cholesky{Float64,Array{Float64,2}}
U factor:
16384×16384 UpperTriangular{Float64,Array{Float64,2}}:
 1.0  0.035674  0.035674    0.00896842  …  1.0389e-259   9.8845e-261 
  ⋅   0.999363  0.00770069  0.0353766      1.0787e-258   1.03603e-259
  ⋅    ⋅        0.999334    0.035105       1.09086e-258  1.02808e-259
  ⋅    ⋅         ⋅          0.998717       1.15209e-257  1.09619e-258
  ⋅    ⋅         ⋅           ⋅             1.10363e-257  1.07026e-258
  ⋅    ⋅         ⋅           ⋅          …  1.13635e-256  1.11247e-257
  ⋅    ⋅         ⋅           ⋅             1.19073e-256  1.14356e-257
  ⋅    ⋅         ⋅           ⋅             1.23683e-255  1.19944e-256
  ⋅    ⋅         ⋅           ⋅             1.14245e-257  1.06657e-258
  ⋅    ⋅         ⋅           ⋅             1.20984e-256  1.14012e-257
  ⋅    ⋅         ⋅           ⋅          …  1.19319e-256  1.10363e-257
  ⋅    ⋅         ⋅           ⋅             1.28447e-255  1.19935e-256
  ⋅    ⋅         ⋅           ⋅             1.26517e-255  1.2

In [21]:
include("hchol_sh.jl")
Σ1 = copy(Σ)
#println(@elapsed cholesky(Σ))
println(@elapsed L = hchol(Symmetric(Σ), n))
norm(uncompress(L, n) - Σ1, 2)

0.755315701


0.17948764113610485

In [22]:
norm(uncompress(L, n) - Σ1, 2)/norm(Σ1, 2)

0.07632712063245985

In [11]:
using Pkg
Pkg.add("LowRankApprox")
using LowRankApprox
@elapsed svd(Σ[1:N÷2, (N÷2 + 1):N])

  Updating registry at `C:\Users\spatial stat\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
 Installed Nullables ───── v1.0.0
 Installed LowRankApprox ─ v0.4.0
  Updating `C:\Users\spatial stat\.julia\environments\v1.2\Project.toml`
  [898213cb] + LowRankApprox v0.4.0
  Updating `C:\Users\spatial stat\.julia\environments\v1.2\Manifest.toml`
  [898213cb] + LowRankApprox v0.4.0
  [4d1e1d77] + Nullables v1.0.0


┌ Info: Precompiling LowRankApprox [898213cb-b102-5a47-900c-97e73b919f73]
└ @ Base loading.jl:1242


9.3270333

In [31]:
nb = N ÷ 2
kmax = 16 + Int(floor(3 * sqrt(nb)))
kmax = kmax < nb ? kmax : nb

opts = LRAOptions(maxdet_tol=0., sketch_randn_niter=1)
opts.sketch = :randn
opts.rtol = 5*eps(real(Float64))
@elapsed U, S, V = psvd(Σ[1:N÷2, (N÷2 + 1):N], opts, rank = n)
norm(U * diagm(S) * V' - Σ[1:N÷2, (N÷2 + 1):N], 2)

4.7448153293613463e-5

In [35]:
size(U * diagm(sqrt.(S)))

(2048, 64)

In [ ]:
function hchol(A::Symmetric{T,Array{T,2}}, m::Int; tol=convert(T, 1e-8)) where T<: AbstractFloat
    A = Matrix(A)
    n = size(A)[1]
    nlev = Int(floor(log2(n/m)))
    for i in 1:nlev
        nb = n ÷ 2^i
        xbegin = 0; ybegin = nb
        for j in 1:2^(i - 1)
            A[(xbegin + 1):(xbegin + nb), (ybegin + 1):(ybegin + nb)]
            xbegin += nb; ybegin += nb
        end
    end
end